In [59]:
from datascience import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from scipy.stats import chi2_contingency
import pandas as pd
import numpy as np
import scipy
import random
import nltk
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize

In [60]:
#Get data
rmp = pd.read_csv("rmf-with-gender.csv")

#Select revelant columns
rmp = rmp[['Overall_Quality', 'Comment', 'Prof_Gender']]

#Drop rows with missing values
rmp = rmp.dropna()

#The mean rating is roughly 3.6, so we'll call everyone who received a rating >= 3 satisfactory
satisfactory = ["yes" if i >= 3 else "no" for i in rmp["Overall_Quality"]]
rmp["Satisfactory"] = satisfactory
rmp = rmp.drop("Overall_Quality", axis=1)

#Remove gender
tokens = [word_tokenize(i) for i in rmp["Comment"]]
gender_pronouns = ["he", "him", "his", "she", "her", "hers", "man", "woman", "himself", "herself"]
no_gender = [[i for i in list if i not in gender_pronouns] for list in tokens]
cleanText = [" ".join(i) for i in no_gender] 
rmp["Comment"] = cleanText

In [61]:
#Descriptive statistics 

#Sample size 
N = len(rmp)

#Gender distribution 
num_f = len(rmp[rmp["Prof_Gender"] == "F"]) 
prop_f = num_f / N #sample is 38% female
sat_f = len(rmp[(rmp["Prof_Gender"]=="F") & (rmp["Satisfactory"]=="yes")])
unsat_f = len(rmp[(rmp["Prof_Gender"]=="F") & (rmp["Satisfactory"]=="no")])

num_m = len(rmp[rmp["Prof_Gender"] == "M"]) 
prop_m = num_m / N #sample is 62% male
sat_m = len(rmp[(rmp["Prof_Gender"]=="M") & (rmp["Satisfactory"]=="yes")])
unsat_m = len(rmp[(rmp["Prof_Gender"]=="M") & (rmp["Satisfactory"]=="no")])

#Chi-square test of independence 
men_women = np.array([[sat_m, unsat_m],[sat_f, unsat_f]])
chi_square = scipy.stats.chi2_contingency(men_women)
p_value = chi_square[1] #Reject null--gender and rating are not independent. P is essentially 0.

In [62]:
#Track gender
track_gender = np.arange(0, N, 1)
rmp["Track_Gender"] = track_gender
gender_match = rmp[["Track_Gender","Prof_Gender"]]

#Drop gender from training data
rmp = rmp.drop("Prof_Gender", axis = 1)

In [63]:
#Split data
XTrain, XTest, yTrain, yTest = train_test_split(rmp["Comment"], rmp[["Satisfactory", "Track_Gender"]], test_size=0.3, random_state=50)

In [64]:
#Tokenize 
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(XTrain)

In [65]:
#TF-IDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [66]:
#Train the classifier 
#I'm choosing to use a logistic classifier because in a previous exercise, it was the most accurate
clf = LogisticRegression().fit(X_train_tfidf, yTrain["Satisfactory"])

In [67]:
#Predict outcomes
X_new_counts = count_vect.transform(XTest)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)

In [68]:
#Build a pipeline
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression()),])

text_clf.fit(XTrain, yTrain["Satisfactory"])  

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [69]:
#Evaluate the classifier
predicted = text_clf.predict(XTest)
np.mean(predicted==np.array(yTest["Satisfactory"]))

0.79903829384358571

In [70]:
#Evaluate the gender bias of of the classifier 
gender_bias = Table.from_df(yTest)
gender_match = Table.from_df(gender_match)

gender_bias = gender_bias.relabeled("Satisfactory", "Actual")
gender_bias = gender_bias.with_column ("Predicted", predicted )
gender_bias = gender_bias.join("Track_Gender", gender_match, "Track_Gender").drop("Track_Gender")

#Select random samples of F and M professors to get equivalent gender distribution
sample_F = gender_bias.where("Prof_Gender", "F").sample(10000)
sample_M = gender_bias.where("Prof_Gender", "M").sample(10000)
gender_50_50 = sample_M.append(sample_F)

#Gender and quality distributions
sat_F = gender_50_50.where("Prof_Gender", "F").where("Predicted", "yes").num_rows
unsat_F = gender_50_50.where("Prof_Gender", "F").where("Predicted", "no").num_rows

sat_M = gender_50_50.where("Prof_Gender", "M").where("Predicted", "yes").num_rows
unsat_M = gender_50_50.where("Prof_Gender", "M").where("Predicted", "no").num_rows

In [71]:
#Chi-square test of independence 
chi_square_50_50 = np.array([[sat_M, sat_F],[unsat_M, unsat_F]])
chi_square_stat = scipy.stats.chi2_contingency(chi_square_50_50)
p_value_50_50 = chi_square_stat[1] #Reject null--gender and rating are not independent at alpha = .05

In [72]:
#Calibration: same likelihood of satisfactory/unsatisfactory for men and women
print("p-value: " + str(p_value_50_50))
print("proportion of satisfactory female teachers: " + str(sat_F/10000))
print("proportion of satisfactory male teachers: " + str(sat_M/10000))

p-value: 0.570776187328
proportion of satisfactory female teachers: 0.9248
proportion of satisfactory male teachers: 0.927


In [73]:
#Error rate balance: false positive and false negative error rates are equal across groups

false_pos_m = gender_50_50.where("Prof_Gender", "M").where("Actual", "yes").where("Predicted", "no").num_rows
false_neg_m = gender_50_50.where("Prof_Gender", "M").where("Actual", "no").where("Predicted", "yes").num_rows
false_pos_rate_m = false_pos_m/10000
false_neg_rate_m = false_neg_m/10000

false_pos_f = gender_50_50.where("Prof_Gender", "F").where("Actual", "yes").where("Predicted", "no").num_rows
false_neg_f = gender_50_50.where("Prof_Gender", "F").where("Actual", "no").where("Predicted", "yes").num_rows
false_pos_rate_f = false_pos_f/10000
false_neg_rate_f = false_neg_f/10000


print("false positve error rate of female teachers: " + str(false_pos_rate_f))
print("false negative error rate of female teachers: " + str(false_neg_rate_f))
print("false positve error rate of male teachers: " + str(false_pos_rate_m))
print("false negative error rate of male teachers: " + str(false_neg_rate_m))

false positve error rate of female teachers: 0.0306
false negative error rate of female teachers: 0.1779
false positve error rate of male teachers: 0.0299
false negative error rate of male teachers: 0.1683


In [16]:
#Data visualization

#Match rmp universities to DOE list of universities, which contains city information
rmp_locations = pd.read_csv("rmf-with-gender.csv")
rmp_locations = rmp_locations[["School", "Overall_Quality", "Comment", "Prof_Gender"]]
rmp_locations = rmp_locations.dropna()
rmp_locations.rename(columns={"School": "Institution_Name"}, inplace=True)

university_roster = pd.read_csv("Accreditation_04_2017.csv")
university_roster = university_roster[["Institution_Name", "Institution_City", "Institution_State", "Institution_Zip"]]

#Create an array of all unique cities
locations = pd.merge(university_roster, rmp_locations, on="Institution_Name", how='inner')
locations = locations["Institution_City"]
locations = locations.unique()

#Plot 
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.basemap import Basemap
from geopy.geocoders import Nominatim
import math
import shapefile

scale = 5

map = Basemap(llcrnrlon=-119,llcrnrlat=22,urcrnrlon=-64,urcrnrlat=49,
        projection='lcc',lat_1=32,lat_2=45,lon_0=-95)

# load the shapefile, use the name 'states'
map.readshapefile("cb_2015_us_state_20m", name='states', drawbounds=True)

# Get the location of each city and plot it
geolocator = Nominatim()

for (city) in locations:
    loc = geolocator.geocode(city)
    x, y = map(loc.longitude, loc.latitude)
    map.plot(x,y,marker='o',color='Red')

plt.show()